## Load the data

Install and import relevant libraries

In [1]:
!conda install folium -c conda-forge
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import folium
import requests
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors
print('done')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.8.0                      py_0    conda-forge
done


Import previously saved Toronto Geospatial CSV

In [2]:
TData = pd.read_csv("torontoData.csv")
TData.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let's explore the data we have;

In [3]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(TData['Borough'].unique()),
        TData.shape[0]
    )
)

Toronto has 11 boroughs and 103 neighborhoods.


Now we need Toronto's coordinates to build the map later;

In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Toronto's coordinates are {}, {}.".format(latitude, longitude))

Toronto's coordinates are 43.653963, -79.387207.


In [5]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = TData

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],
                                           neighborhoods['Longitude'],
                                           neighborhoods['Borough'],
                                           neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Foursqaure configuration

Next, let's configure Foursquare access;

In [6]:
CLIENT_ID = 'xxxxx'
CLIENT_SECRET = 'xxxxx' 
VERSION = '20190303'

Then define a function to get the top 100 venues around each neighbourhood in Toronto;

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's run the function;

In [8]:
Toronto_venues = getNearbyVenues(TData.Neighbourhood,
                            TData.Latitude,
                            TData.Longitude)

100%|██████████| 103/103 [00:28<00:00,  3.66it/s]


check the size of the dataframe and let's see some of it;

In [9]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2239, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


We can see how many unique venues there are;

In [10]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


Let's use one hot encoding on the venue categories;

In [11]:
Toronto_OHE = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_OHE["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_OHE.columns).index("Neighborhood")
cols = deque(Toronto_OHE.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_OHE = Toronto_OHE[cols]

Toronto_OHE.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,...,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We compute the average number of venue categories per neighborhood;

In [12]:
Toronto_grouped = Toronto_OHE.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,...,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue
0,"Adelaide,King,Richmond",0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,"Alderwood,Long Branch",0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


For clustering let us select the 10 most frequent venue categories;
first we define a function to get the N number of top venue categories

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then we run it;

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Breakfast Spot,Restaurant,American Restaurant,Asian Restaurant,Bakery
1,Agincourt,Breakfast Spot,Skating Rink,Lounge,Clothing Store,Music Venue,Antique Shop,Airport Lounge,Airport Service,Airport Terminal,American Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Music Venue
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Pharmacy,Sandwich Place,Fast Food Restaurant,Discount Store,Coffee Shop,Beer Store,Japanese Restaurant,Video Store
4,"Alderwood,Long Branch",Pizza Place,Pub,Pharmacy,Skating Rink,Sandwich Place,Gym,Coffee Shop,Airport Service,Airport Terminal,American Restaurant


## Clustering the neighborhoods

Before clustering, let us use a PCA decomposition to reduce the noise of the signal and improve clustering efficiency

In [15]:
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

Carry out the KMeans clustering

In [16]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[0 0 2 0 0 0 0 0 0 0]
(99,)


Let us now create a dataframe that containes the neighborhood, the location and the cluster information, together with the top 10 venues

In [17]:
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = TData.merge(Toronto_grouped, left_on = "Neighbourhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,"Rouge,Malvern",0.0,0.0,0.0,0.0,...,Print Shop,Fast Food Restaurant,Music Venue,Aquarium,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Airport Gate
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,"Highland Creek,Rouge Hill,Port Union",0.0,0.0,0.0,0.0,...,Golf Course,Bar,History Museum,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,"Guildwood,Morningside,West Hill",0.0,0.0,0.0,0.0,...,Rental Car Location,Medical Center,Breakfast Spot,Electronics Store,Pizza Place,Spa,Mexican Restaurant,American Restaurant,Airport Lounge,Airport Service
3,M1G,Scarborough,Woburn,43.770992,-79.216917,Woburn,0.0,0.0,0.0,0.0,...,Coffee Shop,Korean Restaurant,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Music Venue
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,Cedarbrae,0.0,0.0,0.0,0.0,...,Caribbean Restaurant,Lounge,Bank,Bakery,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Other Great Outdoors,BBQ Joint


In [18]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyze Clusters

As we analyze the clusters next, we will see that the every cluster has Airport services of some sort, so I will ignore that fact and find other common categories in the clusters.

### Cluster 1

In [19]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Print Shop,Fast Food Restaurant,Music Venue,Aquarium,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Airport Gate
1,Golf Course,Bar,History Museum,Music Venue,American Restaurant,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal
2,Rental Car Location,Medical Center,Breakfast Spot,Electronics Store,Pizza Place,Spa,Mexican Restaurant,American Restaurant,Airport Lounge,Airport Service
3,Coffee Shop,Korean Restaurant,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Music Venue
4,Caribbean Restaurant,Lounge,Bank,Bakery,Athletics & Sports,Fried Chicken Joint,Hakka Restaurant,Thai Restaurant,Other Great Outdoors,BBQ Joint


Cluster 1 seems to be dominated by Restaurants.

### Cluster 2

In [20]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Baseball Field,Construction & Landscaping,Music Venue,Antique Shop,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arcade
97,Baseball Field,Music Venue,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery


Cluster 2 is all about baseball fields.

### Cluster 3

In [21]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Convenience Store,Playground,Music Venue,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium
14,Park,Playground,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Music Venue
23,Park,Bank,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Music Venue
25,Park,Fast Food Restaurant,Food & Drink Shop,Bus Stop,Aquarium,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop
30,Park,Bus Stop,Airport,Athletics & Sports,Asian Restaurant,Arts & Crafts Store,Auto Garage,Art Museum,Art Gallery,Airport Food Court


Cluster 3 is all about parks and restaurants.

### Cluster 4

In [22]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Bank,Music Venue,Arcade,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery


Cluster 4 is one neighborhood and it is banking related.